# Programming Lab 1 

## Handout 7 

## Lara Schmalenstroer (s0laschm@uni-bonn.de)

## Read mapping and exact string matching

### 1.(5+3pts) Naive string matching

#### (a) (5pts) Mapping reads

In [12]:
import re

def map_reads_naive(reads,sequence):
    positions={}
    for read in reads:
        pattern=re.compile(read)
        match = re.finditer(pattern,sequence)
        positions[read]=[]
        for m in match:
            pat=m.group()
            pos=m.start()
            positions[read].append(pos)
        if len(positions[read])==0:
            del positions[read]
    return positions

In [13]:
sequence='abracadabra'
reads=['abrac','cadab','racad','adabr']

map_reads_naive(reads,sequence)

{'abrac': [0], 'cadab': [4], 'racad': [2], 'adabr': [5]}

### 2. (6pts) Speed of matching
#### (a) (3pts) Generate random data

In [16]:
import random

def get_random_data(genome_size,nr_of_reads,read_length,random_seed):
    #assert type(genome_size)==int and type(nr_of_reads)==int and type(read_length)==int
    alphabet=['a','c','g','t']
    random.seed(random_seed)
    genome=''.join(random.choices(alphabet,k=genome_size))
    indices=[random.randint(0,genome_size-read_length) for i in range(nr_of_reads)]
    reads=[genome[index:index+read_length] for index in indices]
    return (genome,reads)

In [17]:
get_random_data(10,3,4,2)

('ttaatggcgg', ['tggc', 'ggcg', 'taat'])

#### (b) (3pts) Test performance of read mapping
##### (i)

In [18]:
import timeit
genome,reads=get_random_data(1000000, 10000,10,2)
start=timeit.default_timer()
map_reads_naive(reads,genome)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 60.15490810000006s


##### (ii)

In [19]:
genome,reads=get_random_data(1000000, 100000,11,2)
start=timeit.default_timer()
map_reads_naive(reads,genome)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 596.0370452s


##### (iii)

In [41]:
genome,reads=get_random_data(10000000, 10000,12,2)
start=timeit.default_timer()
map_reads_naive(reads,genome)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 564.8129436999998s


### 3. (9pts) Preprocessing the "genome"
#### (a) (6pts) Implement the read mapping function map_reads_lookup(reads,sequence,lookup_size) using the outlined strategy. Here, lookup_size is the size of the substrings to be used in the lookup table.

In [20]:
def preprocessing(genome,k):
    positions={}
    ind=0
    kmer=genome[ind:ind+k]
    while len(kmer)==k:
        if kmer not in positions.keys():
            positions[kmer]=[ind]
        else:
            positions[kmer].append(ind)
        ind+=1
        kmer=genome[ind:ind+k]
    return positions

In [21]:
preprocessing(sequence,4)

{'abra': [0, 7],
 'brac': [1],
 'raca': [2],
 'acad': [3],
 'cada': [4],
 'adab': [5],
 'dabr': [6]}

In [22]:
test='acgtatcgtatcc'
res=preprocessing(test,4)

In [23]:
res['acgt']

[0]

In [24]:
def map_reads_lookup(reads,genome,lookup_size):
    positions=preprocessing(genome,lookup_size)
    matches={}
    for read in reads:
        matches[read]=[]
        k=read[:lookup_size]
        indices=positions[k]
        for index in indices:
            if genome[index:index+len(read)]==read:
                matches[read].append(index)
    return matches

#### (b) (3pts) Use a lookup size of 4 and 8 and determine the running times for the sample data generated in ex. 2(b).
##### (i)

In [25]:
import timeit
genome,reads=get_random_data(1000000, 10000,10,2)
start=timeit.default_timer()
map_reads_lookup(reads,genome,4)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 25.09676790000003s


In [26]:
genome,reads=get_random_data(1000000, 10000,10,2)
start=timeit.default_timer()
map_reads_lookup(reads,genome,8)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 2.2620523999999023s


##### (ii)

In [27]:
genome,reads=get_random_data(1000000, 100000,11,2)
start=timeit.default_timer()
map_reads_lookup(reads,genome,4)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 236.42717119999998s


In [28]:
genome,reads=get_random_data(1000000, 100000,11,2)
start=timeit.default_timer()
map_reads_lookup(reads,genome,8)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 4.032806900000196s


##### (iii)

In [29]:
genome,reads=get_random_data(10000000, 10000,12,2)
start=timeit.default_timer()
map_reads_lookup(reads,genome,4)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 248.5499852999999s


In [30]:
genome,reads=get_random_data(10000000, 10000,12,2)
start=timeit.default_timer()
map_reads_lookup(reads,genome,8)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 24.29284369999982s


### 4. (10+3pts) Preprocessing the reads
#### (a) (4pts) Write a Python class KeywordTree for representing such a keyword tree. The class will need to contain at least one method add_word for adding new words to a tree.

In [31]:
class KeywordTree:
    def __init__(self):
        self.root={}
    
    def AddWord(self,word):
        first=self.root
        for letter in word:
            if letter not in first:
                first[letter]={}
                if word.endswith(letter):
                    first[letter]={'word':'word'}
            first=first[letter]
        return first

In [32]:
sequence='abracadabra'
reads=['abrac','cadab','racad','adabr']

In [33]:
def make_keyword_tree(reads):
    tree=KeywordTree()
    for read in reads:
        tree.AddWord(read)
    return tree.root

In [34]:
dict_tree=make_keyword_tree(reads)

In [35]:
dict_tree

{'a': {'b': {'r': {'a': {'c': {'word': 'word'}}}},
  'd': {'a': {'b': {'r': {'word': 'word'}}}}},
 'c': {'a': {'d': {'a': {'b': {'word': 'word'}}}}},
 'r': {'a': {'c': {'a': {'d': {'word': 'word'}}}}}}

#### (b) (3pts) Write a read mapping function map_reads_tree(reads_tree,sequence), which uses keyword trees for read mapping. Here, the argument reads_tree should be the keyword tree generated from the reads. The function, again, should return a dictionary as described in ex.1 (a).

In [36]:
def map_reads_tree(reads_tree,sequence):
    mapping={}
    for i in range(len(sequence)):
        dic_reads=reads_tree
        letter=sequence[i]
        word=letter
        j=i
        while letter in dic_reads.keys():
            if dic_reads[letter]=={'word':'word'}:
                if word in mapping.keys():
                    mapping[word].append(i)
                else:
                    mapping[word]=[i]
                break
            dic_reads=dic_reads[letter]
            if j!=len(sequence)-1:
                j+=1
            letter=sequence[j]
            word+=letter
    return mapping

In [37]:
map_reads_tree(dict_tree,sequence)

{'abrac': [0], 'racad': [2], 'cadab': [4], 'adabr': [5]}

#### (d) (3pts) Determine the running times for the sample data generated in ex. 2 (b).
##### (i)

In [38]:
import timeit
genome,reads=get_random_data(1000000, 10000,10,2)
tree=make_keyword_tree(reads)
start=timeit.default_timer()
map_reads_tree(tree,genome)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 9.281505900000411s


##### (ii)

In [39]:
genome,reads=get_random_data(1000000, 100000,11,2)
tree=make_keyword_tree(reads)
start=timeit.default_timer()
map_reads_tree(tree,genome)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 14.799783399999797s


##### (iii)

In [40]:
genome,reads=get_random_data(10000000, 10000,12,2)
tree=make_keyword_tree(reads)
start=timeit.default_timer()
map_reads_tree(tree,genome)
stop=timeit.default_timer()
t=stop-start
print(f'Time: {t}s')

Time: 95.59415820000004s
